# Проект анализа ДТП  
**Источник данных:** http://stat.gibdd.ru/

Дорожно-транспортные происшествия (ДТП) являются одной из основных причин травматизма и смертности на дорогах. Частота аварий зависит от множества факторов: времени суток, дня недели, опыта водителя, типа транспортного средства и особенностей региона. Цель проекта — выявить закономерности, позволяющие прогнозировать число ДТП и использовать эти знания для профилактики.

Я использую методы линейной регрессии для моделирования и объяснения количества ДТП в зависимости от временных факторов. Модель поможет дорожным службам предсказывать периоды повышенного риска и своевременно усиливать меры безопасности.

---

## Задача:
С помощью **линейной регрессии** получить модель, предсказывающую число ДТП в зависимости от дня недели, времени суток, типа транспорта и других параметров.

**Цель:** Получить интерпретируемую модель, отражающую влияние дня недели на частоту ДТП.